In [ ]:
import os, json, pickle, re, hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import make_interp_spline
import tqdm

DAY_LIMIT = 10

# --- Settings ---
CITIES = ["ISTANBUL"]
cityname_map = {
    "ISTANBUL": "Istanbul",
    "LONDON": "London",
    "NY": "New York"
}

city_colors = {
    "ISTANBUL": "#FFC300",
    "LONDON": "#FF5733",
    "NY": "#33AFFF"
}
timezone_map = {
    "ISTANBUL": pytz.timezone("Europe/Istanbul"),
    "LONDON": pytz.timezone("Europe/London"),
    "NY": pytz.timezone("America/New_York")
}

highwayMap = {'motorway':0, 'highway':1, 'primary':2, 'secondary':3, 'tertiary':4, 'unclassified':5, 'residential':6, 'living_street': 7}
theta = [100, 50, 20, 17, 10, 8, 8, 5]

# --- Helpers ---
def generate_shape_key(shape):
    shape_str = json.dumps(shape, sort_keys=True)
    return hashlib.md5(shape_str.encode()).hexdigest()

def process_traffic_snapshot(traffic_data, roadKey_to_osmid, osmid_to_capacity):
    flows, capacities = [], []
    for seg in traffic_data['results']:
        shape_data = seg['location']['shape']
        jam_factor = seg['currentFlow']['jamFactor']
        key = generate_shape_key(shape_data)
        osmids_for_key = roadKey_to_osmid.get(key, [])
        if not isinstance(osmids_for_key, list):
            osmids_for_key = [osmids_for_key]
        for osm_id in osmids_for_key:
            capacity = osmid_to_capacity.get(osm_id, 0)
            flows.append((jam_factor / 10.0) * capacity)
            capacities.append(capacity)
    flows, capacities = np.array(flows), np.array(capacities)
    return float(np.sum(flows / capacities) / len(flows)) if len(flows) > 0 else 0.0

def parse_timestamp_from_filename(filename):
    match = re.search(r"(?:traffic_data_|traffic__|traffic_)(\d{8})_(\d{6})\.json", filename)
    if not match:
        raise ValueError(f"Filename format not recognized: {filename}")
    date_str, time_str = match.group(1), match.group(2)
    dt = datetime.strptime(date_str + time_str, "%Y%m%d%H%M%S")
    return dt.isoformat() + 'Z'

# --- Main Loop ---
city_hourly_data = {}

for city in CITIES:
    print(f"Processing {city}")
    PATH = city
    flow_folder = os.path.join(PATH, "realFlowData")
    if not os.path.exists(flow_folder):
        print(f"Missing data folder for {city}")
        continue

    try:
        with open(os.path.join(PATH, "roadKey_to_osmid.pkl"), "rb") as f:
            roadKey_to_osmid = pickle.load(f)
        with open(os.path.join(PATH, "osmnx_subgraph.pkl"), "rb") as f:
            subG = pickle.load(f)
    except Exception as e:
        print(f"Missing graph data for {city}: {e}")
        continue

    # Set capacity
    for edge in subG.edges:
        highway = subG.edges[edge].get("highway", "unclassified")

        # Handle case where highway is a list like ['residential', 'unclassified']
        if isinstance(highway, list):
            highway = highway[0]  # Take the first type as primary

        if highway not in highwayMap:
            highway = "unclassified"

        subG.edges[edge]["capacity"] = subG.edges[edge]["length"] * theta[highwayMap[highway]]


    # Build capacity map
    osmid_to_capacity = {}
    for u, v, k, data in subG.edges(keys=True, data=True):
        osmids = data.get('osmid', [])
        if not isinstance(osmids, list):
            osmids = [osmids]
        for osm_id in osmids:
            osmid_to_capacity[osm_id] = data.get("capacity", 0)

    flow_values = {}
    processed_dates = set()

    for fname in tqdm.tqdm(os.listdir(flow_folder)):
        if not fname.endswith(".json"):
            continue
        try: 
            date = fname.split("_")[1]
            processed_dates.add(date)
            
            if(len(processed_dates)) > DAY_LIMIT:
                break
            
            timestamp = parse_timestamp_from_filename(fname)

            with open(os.path.join(flow_folder, fname), "r") as f:
                td = json.load(f)
            
            flow_values[timestamp] = process_traffic_snapshot(td, roadKey_to_osmid, osmid_to_capacity)
        except Exception as e:
            print(f"Error processing {fname}: {e}")


    # Timezone conversion
    local_tz = timezone_map[city]
    sorted_keys = sorted(flow_values.keys())
    sorted_datetimes = [datetime.strptime(ts, '%Y-%m-%dT%H:%M:%SZ') for ts in sorted_keys]
    local_times = [pytz.utc.localize(dt).astimezone(local_tz) for dt in sorted_datetimes]
    sorted_values = [flow_values[k] for k in sorted_keys]

    # Hourly averaging
    df = pd.DataFrame({"timestamp": local_times, "value": sorted_values})
    df["hour"] = pd.to_datetime(df["timestamp"]).dt.hour
    hourly_avg = df.groupby("hour")["value"].mean().reset_index()

    # Normalize
    scaler = MinMaxScaler()
    hourly_avg["normalized_value"] = scaler.fit_transform(hourly_avg[["value"]])
    city_hourly_data[city] = hourly_avg

In [ ]:
df_raw = pd.DataFrame({"timestamp": local_times, "value": sorted_values})
df_raw["timestamp"] = pd.to_datetime(df_raw["timestamp"])
df_raw = df_raw.set_index("timestamp")

df_5min = df_raw["value"].resample("5T").mean().dropna().reset_index()
df_5min["hour_fraction"] = df_5min["timestamp"].dt.hour + df_5min["timestamp"].dt.minute / 60.0

x_fit_base = df_5min["hour_fraction"].values
y_fit_base = df_5min["value"].values

poly = np.poly1d(np.polyfit(x_fit_base, y_fit_base, deg=23))

# Generate 5-min resolution curve for plotting/comparison
x_curve = np.arange(0, 24, 5 / 60.0)
y_curve = poly(x_curve)

df_hour = df_raw.resample("1H").mean().dropna().reset_index()
df_hour["hour_fraction"] = df_hour["timestamp"].dt.hour + df_hour["timestamp"].dt.minute / 60.0

df_10min = df_raw.resample("10T").mean().dropna().reset_index()
df_10min["hour_fraction"] = df_10min["timestamp"].dt.hour + df_10min["timestamp"].dt.minute / 60.0


plt.figure(figsize=(10, 5))
plt.plot(df_5min["hour_fraction"], df_5min["value"], '.', label="5-min Data", alpha=0.4)
plt.plot(df_hour["hour_fraction"], df_hour["value"], 'o', label="Hourly Avg", markersize=5)
plt.plot(df_10min["hour_fraction"], df_10min["value"], 's', label="10-min Avg", markersize=4)
plt.plot(x_curve, y_curve, '-', linewidth=2, label="Polynomial Fit (5-min Base)")
plt.xlabel("Hour of Day")
plt.ylabel("Jam Factor")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

fitted_models = {}

for city, df in city_hourly_data.items():
    x = df['hour'].values
    y = df['value'].values
    coeffs = np.polyfit(x, y, deg=4)  # or deg=1 or deg=5 depending on your needs
    fitted_models[city] = np.poly1d(coeffs)

    # Save the coefficients to file
    folder = os.path.join(city)
    os.makedirs(folder, exist_ok=True)  # ensure folder exists
    np.save(os.path.join(folder, "jam_model_poly.npy"), coeffs)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get data
df = city_hourly_data["ISTANBUL"]
x = df['hour'].values
y = df['value'].values

# Fit 3rd degree polynomial
coeffs = np.polyfit(x, y, deg=4)
poly = np.poly1d(coeffs)

# Evaluate fit
x_fit = np.linspace(0, 23, 500)
y_fit = poly(x_fit)

# Plot
plt.figure(figsize=(8, 4))
plt.plot(x, y, 'o', label='Original Data')
plt.plot(x_fit, y_fit, '-', label='3rd Degree Polynomial Fit')
plt.xlabel("Hour")
plt.ylabel("Normalized Jam Factor")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

fitted_models = {}

for city, df in city_hourly_data.items():
    x = df['hour'].values
    y = df['value'].values
    coeffs = np.polyfit(x, y, deg=1)
    fitted_models[city] = np.poly1d(coeffs)

for city, poly_model in fitted_models.items():
    folder = os.path.join(city)
    np.save(os.path.join(folder, "jam_model_poly.npy"), poly_model.coefficients)


In [ ]:
import numpy as np

# Load the coefficients array from the .npy file
coeffs = np.load(PATH + "/jam_model_poly.npy")

# Create a polynomial function from coefficients
poly_model = np.poly1d(coeffs)

# Now use it like a function
hour = 13.5
predicted_value = poly_model(hour)
print(f"Predicted jam factor at hour {hour}: {predicted_value}")


In [ ]:
import scripts.TrafficModelWithJunctionRespect as tmjr
import importlib
importlib.reload(tmjr)

# D3M
sim = tmjr.simulator(G, np.array(theta), highwayMap)
sim.simulate(np.array(M), 100, True, jamModel=poly_model)